In [0]:
#Bidirectional LSTM Model

#importing all libraries and modules required 
import numpy as np
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional, TimeDistributed

#function to convert 3-format input data to 5-format
def convert_data(data,max_len):
	"""Pad the data to be stroke-5 bigger format as described in paper."""
	start_stroke_token = [0, 0, 1, 0, 0]
	result = np.zeros((data.shape[0], max_len + 1, 5), dtype=float)
	for i in range(data.shape[0]):
		l = len(data[i])
		assert l <= max_len
		result[i, 0:l, 0:2] = data[i][:, 0:2]
		result[i, 0:l, 3] = data[i][:, 2]
		result[i, 0:l, 2] = 1 - result[i, 0:l, 3]
		result[i, l:, 4] = 1
		# put in the first token, as described in sketch-rnn methodology
		result[i, 1:, :] = result[i, :-1, :]
		result[i, 0, :] = 0
		result[i, 0, 2] = start_stroke_token[2]  # setting S_0 from paper.
		result[i, 0, 3] = start_stroke_token[3]
		result[i, 0, 4] = start_stroke_token[4]
	return result

#load the datset
folder = "/content/drive/My Drive/AML/"
with np.load(folder+"airplane.npz",encoding='latin1') as data:
	train_data = data['train']
	valid_data = data['valid']	
	test_data = data['test']

#training parameters
batch_size = 250
epochs = 30

# hyperparmeters of the model
num_encoder_tokens = 5
num_decoder_tokens = 5
intermediate_dim = 128

#find max_seq_length in the data
max_seq_len = 0
all_data = np.concatenate((train_data,valid_data,test_data))
for instance in all_data:
	shape = np.shape(instance)
	if shape[0] > max_seq_len:
		max_seq_len  = shape[0]

print("Max Sequence Length:",max_seq_len)

#convert 3-format data to 5-format
train_data = convert_data(train_data, max_seq_len)
test_data = convert_data(test_data, max_seq_len)
valid_data = convert_data(valid_data, max_seq_len)

# Define an input sequence and process it.
input_layer = Input(shape=(None, num_encoder_tokens), name = "input_layer")
lstm_layer = Bidirectional(LSTM(intermediate_dim, return_sequences=True, name = "LSTM_layer"))
lstm_outputs = lstm_layer(input_layer)
decoder_dense = TimeDistributed(Dense(num_decoder_tokens, activation='relu', name = "output_layer"))
decoder_outputs = decoder_dense(lstm_outputs)
model = Model([input_layer], decoder_outputs)

#compile
model.compile(optimizer='rmsprop',loss = 'mean_squared_error', metrics=['accuracy'])

#train
model.fit(train_data[:,:max_seq_len,:], train_data[:,1:max_seq_len+1,:],
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(valid_data[:,:max_seq_len,:], valid_data[:,1:max_seq_len+1,:]))

#test
print(model.evaluate(test_data[:,:max_seq_len,:],test_data[:,1:max_seq_len+1,:],batch_size=batch_size))

  
#save the model
model.save(folder+"model/bidirectional_lstm.h5")
print("Saved model to disk")

In [0]:
#LSTM Autoencoder Model (Sequence to Sequence)

#importing all libraries and modules required 
import numpy as np
from keras.models import Model
from keras import backend as K
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Lambda
from keras.layers import Add, Activation
from keras.losses import mse, kullback_leibler_divergence

#function to convert 3-format input data to 5-format
def convert_data(data,max_len):
	"""Pad the data to be stroke-5 bigger format as described in paper."""
	start_stroke_token = [0, 0, 1, 0, 0]
	result = np.zeros((data.shape[0], max_len + 1, 5), dtype=float)
	for i in range(data.shape[0]):
		l = len(data[i])
		assert l <= max_len
		result[i, 0:l, 0:2] = data[i][:, 0:2]
		result[i, 0:l, 3] = data[i][:, 2]
		result[i, 0:l, 2] = 1 - result[i, 0:l, 3]
		result[i, l:, 4] = 1
		# put in the first token, as described in sketch-rnn methodology
		result[i, 1:, :] = result[i, :-1, :]
		result[i, 0, :] = 0
		result[i, 0, 2] = start_stroke_token[2]  # setting S_0 from paper.
		result[i, 0, 3] = start_stroke_token[3]
		result[i, 0, 4] = start_stroke_token[4]
	return result

#load the dataset
folder = "/content/drive/My Drive/AML/"
with np.load(folder+"cat.npz",encoding='latin1') as data:
	train_data = data['train']
	valid_data = data['valid']	
	test_data = data['test']

#training parameters
batch_size = 250
epochs = 100

# configure
num_encoder_tokens = 5
num_decoder_tokens = 5
intermediate_dim = 256

#find max_seq_length in the data
max_seq_len = 0
all_data = np.concatenate((train_data,valid_data,test_data))
for instance in all_data:
	shape = np.shape(instance)
	if shape[0] > max_seq_len:
		max_seq_len  = shape[0]

print("Max Sequence Length:",max_seq_len)

#convert 3-format data to 5-format
train_data = convert_data(train_data, max_seq_len)
test_data = convert_data(test_data, max_seq_len)
valid_data = convert_data(valid_data, max_seq_len)

# Define the model.
encoder_inputs = Input(shape=(None, num_encoder_tokens), name = "encoder_input")
encoder = LSTM(intermediate_dim, return_sequences=True, return_state=True, name = "encoder_lstm")
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens), name = "decoder_input")
decoder_lstm = LSTM(intermediate_dim, return_sequences=True, return_state=True, name = "decoder_lstm")
decoder_output,_,_ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='relu', name = "output_layer")
decoder_outputs = decoder_dense(decoder_output)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#compile
model.compile(optimizer='rmsprop',loss = 'mean_squared_error', metrics=['accuracy'])


#train
model.fit([train_data[:,:max_seq_len,:],train_data[:,:max_seq_len,:]], train_data[:,1:max_seq_len+1,:],
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=([valid_data[:,:max_seq_len,:], valid_data[:,:max_seq_len,:]], valid_data[:,1:max_seq_len+1,:]))

#test
print(model.evaluate([test_data[:,:max_seq_len,:], test_data[:,:max_seq_len,:]],test_data[:,1:max_seq_len+1,:],batch_size=batch_size))

#save the model  
model.save(folder+"model/autoencoder_lstm.h5")
print("Saved model to disk")

In [0]:
#Variational Autoencoder (Sequence to Sequence)

#importing all libraries and modules required 
import numpy as np
from keras.models import Model
from keras import backend as K
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Lambda
from keras.losses import mse, kullback_leibler_divergence

#function to convert 3-format input data to 5-format
def convert_data(data,max_len):
	"""Pad the data to be stroke-5 bigger format as described in paper."""
	start_stroke_token = [0, 0, 1, 0, 0]
	result = np.zeros((data.shape[0], max_len + 1, 5), dtype=float)
	for i in range(data.shape[0]):
		l = len(data[i])
		assert l <= max_len
		result[i, 0:l, 0:2] = data[i][:, 0:2]
		result[i, 0:l, 3] = data[i][:, 2]
		result[i, 0:l, 2] = 1 - result[i, 0:l, 3]
		result[i, l:, 4] = 1
		# put in the first token, as described in sketch-rnn methodology
		result[i, 1:, :] = result[i, :-1, :]
		result[i, 0, :] = 0
		result[i, 0, 2] = start_stroke_token[2]  # setting S_0 from paper.
		result[i, 0, 3] = start_stroke_token[3]
		result[i, 0, 4] = start_stroke_token[4]
	return result

#load the dataset
folder = "/content/drive/My Drive/AML/"
with np.load(folder+"cat.npz",encoding='latin1') as data:
	train_data = data['train']
	valid_data = data['valid']	
	test_data = data['test']

batch_size = 250
epochs = 100

# configure
num_encoder_tokens = 5
num_decoder_tokens = 5
intermediate_dim = 256
latent_dim = 5

#find max_seq_length in the data
max_seq_len = 0
all_data = np.concatenate((train_data,valid_data,test_data))
for instance in all_data:
	shape = np.shape(instance)
	if shape[0] > max_seq_len:
		max_seq_len  = shape[0]

print("Max Sequence Length:",max_seq_len)

#conert 3-format data to 5-format
train_data = convert_data(train_data, max_seq_len)
test_data = convert_data(test_data, max_seq_len)
valid_data = convert_data(valid_data, max_seq_len)

# Define the model.
encoder_inputs = Input(shape=(None, num_encoder_tokens), name="encoder_input")
encoder = LSTM(intermediate_dim, return_sequences=True, return_state=True, name="encoder_lstm")
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


z_mean = Dense(latent_dim)(encoder_outputs)
z_log_var = Dense(latent_dim, activation='relu')(encoder_outputs)

def  sampling(args):
	z_mean, z_log_var = args
	batch = K.shape(z_mean)[0]
	timesteps = K.shape(z_mean)[1]
	dim = K.int_shape(z_mean)[2]
	epsilon = K.random_normal(shape = (batch, timesteps, dim), mean = 0.)
	return z_mean + K.exp(z_log_var/2) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

decoder_inputs = Input(shape = (None,num_decoder_tokens), name = "decoder_input")
decoder_lstm = LSTM(intermediate_dim, return_sequences=True, return_state=True, name = "decoder_lstm")
decoder_output,_,_ = decoder_lstm(z, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='relu')
decoder_outputs = decoder_dense(decoder_output)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#loss function
def get_loss(encoder_inputs, decoder_outputs):
	mse_loss = mse(encoder_inputs, decoder_outputs)
	kl_loss = kullback_leibler_divergence(encoder_inputs, decoder_outputs)
	return mse_loss + kl_loss

#compile
model.compile(optimizer='adam',loss = get_loss, metrics=['accuracy'])


#train
model.fit([train_data[:,:max_seq_len,:], train_data[:,:max_seq_len,:]], train_data[:,1:max_seq_len+1,:],
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=([valid_data[:,:max_seq_len,:], valid_data[:,:max_seq_len,:]], valid_data[:,1:max_seq_len+1,:]))

#test
print(model.evaluate([test_data[:,:max_seq_len,:], test_data[:,:max_seq_len,:]],test_data[:,1:max_seq_len+1,:],batch_size=batch_size))


#save the whole model  
model.save(folder+"model/VAE.h5")

#define encoder model
encoder_model = Model(encoder_inputs, encoder_states)

#define decoder model
decoder_state_input_h = Input(shape=(intermediate_dim,))
decoder_state_input_c = Input(shape=(intermediate_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

sampling_inputs = Input(shape=(latent_dim,))
decoder_outputs, state_h, state_c = decoder_lstm(
    sampling_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([sampling_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

#save encoder and decoder models
encoder_model.save(folder+"model/VAEencoder.h5")
decoder_model.save(folder+"model/VAEdecoder.h5")